In [19]:
# Fix: corrected index usage in time-weighted objective function and flow accumulation terms

import gurobipy as gp
from gurobipy import GRB

# Time and demand settings
tau = 0.25
T = 60.0
N = int(T / tau)
D_EV = 36.0
D_ICV = 36.0

# Flow parameters
V = 900.0
W = 450.0
K = 0.12
Q_a = K * V * W / (V + W)

# Link lengths
length = {1: 0.0, 2: 900.0, 3: 1800.0, 4: 900.0, 5: 3600.0, 6: 900.0, 7: 0.0}
travelV = {a: int((length[a]/V)/tau + 1e-9) for a in length} #dividiamo per tau così invece che avere il tempo di percorrrenza abbiamo quanti step ci vogliono per percorrere
travelW = {a: int((length[a]/W)/tau + 1e-9) for a in length}

# Battery parameters
omega = 0.33
epsilon = 1.8e-4
B_max = 0.5
B0 = 0.35

# Budget constraint
I_budget = 100.0

# Network topology
E = [1, 2, 3, 4, 5, 6, 7]
E_R, E_S, E_A = [1], [7], [2, 3, 4, 5, 6]
pred = {1: [], 2: [1], 3: [1], 4: [2], 5: [2], 6: [3, 4], 7: [5, 6]}
succ = {1: [2, 3], 2: [4, 5], 3: [6], 4: [6], 5: [7], 6: [7], 7: []}

# EV paths
paths = {0: [2, 4, 6], 1: [2, 5], 2: [3, 6]}
delta = {(a, p): int(a in paths[p]) for p in paths for a in E_A}
count_paths = {a: sum(delta[(a, p)] for p in paths) for a in E_A}

model = gp.Model("Dynamic_WCL")
model.setParam("OutputFlag", 0)

x = model.addVars(E_A, vtype=GRB.BINARY, name="x")
y = model.addVars(len(paths), vtype=GRB.BINARY, name="y")
B = {(a, p): model.addVar(ub=B_max, name=f"B_{a}_{p}") for p in paths for a in paths[p]}

classes = ['EV', 'ICV']
u, v, n, f = {}, {}, {}, {}
for m in classes:
    for a in E:
        for i in range(1, N+1):
            u[(m, a, i)] = model.addVar(lb=0.0)
            v[(m, a, i)] = model.addVar(lb=0.0)
            n[(m, a, i)] = model.addVar(lb=0.0)
        for b in pred[a]:
            for i in range(1, N+1):
                f[(m, b, a, i)] = model.addVar(lb=0.0)

alpha = model.addVars(E_A, classes, range(1, N+1), lb=0.0, ub=1.0, name="alpha")
for a in E_A:
    for i in range(1, N+1):
        model.addConstr(gp.quicksum(alpha[a, m, i] for m in classes) == 1.0)

# Corrected time-weighted objective function (Eq. 1)
model.setObjective(gp.quicksum((N - i + 1) * f[(m, b, 7, i)] for m in classes for b in pred[7] for i in range(1, N+1)), GRB.MAXIMIZE)

model.addConstr(gp.quicksum(length[a] * x[a] for a in E_A) <= I_budget)

M = 1e3
#constraints energetiche 4-6
for p, link_list in paths.items():
    for idx, a in enumerate(link_list):
        prevB = B0 if idx == 0 else B[(link_list[idx - 1], p)]
        recharge = omega * (length[a] / V)
        consume = epsilon * length[a]
        model.addConstr(B[(a, p)] <= B_max)
        model.addConstr(B[(a, p)] <= prevB - consume + recharge * x[a] + M * (1 - y[p]))
        model.addConstr(B[(a, p)] >= prevB - consume + recharge * x[a] - M * (1 - y[p]))
        model.addConstr(B[(a,p)]>= M* (y[p]-1))
#è il minimo?

#15 e 18
for i in range(1, N+1):
    model.addConstr(u[('EV', 1, i)] == D_EV)
    model.addConstr(u[('ICV', 1, i)] == D_ICV)
    model.addConstr(v[('EV', 7, i)] == 0)
    model.addConstr(v[('ICV', 7, i)] == 0)
#16 e 17
for m in classes:
    for a in E:
        for i in range(1, N+1):
            if a not in E_R:
                model.addConstr(u[(m, a, i)] == gp.quicksum(f[(m, b, a, i)] for b in pred[a]))
            if a not in E_S:
                model.addConstr(v[(m, a, i)] == gp.quicksum(f[(m, a, b, i)] for b in succ[a]))
#13 e 14
for m in classes:
    for a in E:
        for i in range(1, N+1):
            inflow = gp.quicksum(u[(m, a, k)] for k in range(1, i+1))  # 12
            outflow = gp.quicksum(v[(m, a, k)] for k in range(1, i+1))
            model.addConstr(n[(m, a, i)] == inflow - outflow)
            if a in E_A:
                if i >= travelV[a]:
                    model.addConstr(n[(m, a, i)] >= gp.quicksum(u[(m, a, k)] for k in range(i - travelV[a] + 1, i+1)))
                if i >= travelW[a]:
                    model.addConstr(n[(m, a, i)] + gp.quicksum(v[(m, a, k)] for k in range(i - travelW[a] + 1, i+1)) <= K * length[a] * alpha[a, m, i])
#20-23
for a in E_A:
    for i in range(1, N+1):
        inflow_s = gp.quicksum(u[(m, a, k)] for m in classes for k in range(1, i))
        outflow_s = gp.quicksum(v[(m, a, k)] for m in classes for k in range(1, i - travelW[a] + 1)) if i > travelW[a] else 0
        supply = K * length[a] + outflow_s - inflow_s
        model.addConstr(gp.quicksum(u[(m, a, i)] for m in classes) <= Q_a)
        model.addConstr(gp.quicksum(u[(m, a, i)] for m in classes) <= supply)

        inflow_d = gp.quicksum(v[(m, a, k)] for m in classes for k in range(1, i))
        outflow_d = gp.quicksum(u[(m, a, k)] for m in classes for k in range(1, i - travelV[a] + 1)) if i > travelV[a] else 0
        demand = outflow_d - inflow_d
        model.addConstr(gp.quicksum(v[(m, a, i)] for m in classes) <= Q_a)
        model.addConstr(gp.quicksum(v[(m, a, i)] for m in classes) <= demand)
#19
for m in classes:
    for a in E_A:
        for i in range(1, N+1):
            inflow = gp.quicksum(f[(m, b, a, i)] for b in pred[a])
            if m == 'EV':
                model.addConstr(inflow <= Q_a * gp.quicksum(delta[(a, p)] * y[p] for p in paths))
            else:
                model.addConstr(inflow <= Q_a * count_paths[a])

model.optimize()


In [20]:
# === Output: WCL installation and EV path choices ===
if model.status == GRB.OPTIMAL or model.status == GRB.SUBOPTIMAL:
    print(f"🎯 Valore funzione obiettivo (outflow pesato): {model.ObjVal:.2f}")
    for p in paths:
         for a in paths[p]:
             if B[(a,p)].x > 1e-4:
                 print(f"energia B[({a},{p})]={B[(a,p)]}")
    
    print("🔌 Link selezionati per l'installazione delle WCL:")
    for a in E_A:
        if x[a].x > 0.5:
            print(f"  - Link {a} (lunghezza: {length[a]} m)")

    print("🚗 Percorsi selezionati per EV:")
    for p in paths:
        if y[p].x > 0.5:
            print(f"  - Path {p}: 1 -> " + " -> ".join(str(a) for a in paths[p]) + " -> 7")
else:
    print("⚠️ Nessuna soluzione ottima trovata.")
print(f"energia B[({5},{1})]={B[(5,1)]}")


🎯 Valore funzione obiettivo (outflow pesato): 465840.00
energia B[(2,0)]=<gurobi.Var B_2_0 (value 0.16199999999999348)>
energia B[(3,2)]=<gurobi.Var B_3_2 (value 0.025999999999953616)>
🔌 Link selezionati per l'installazione delle WCL:
🚗 Percorsi selezionati per EV:
energia B[(5,1)]=<gurobi.Var B_5_1 (value 0.0)>
